In [543]:
import os
import csv
import subprocess as sub
#Me ubico en el directorio que quiero 
os.chdir('/home/hoseki/Desktop/Sysbio/cSTING/rot_cgamp/production/hbonds') 
#En el directorio debo tener la lista de residuos que quiero analizar.
#El archivo debe tener los residuos uno abajo del otro, de la forma NNN, sin ningun otro caracter.

#Armo la lista con los residuos que se van a analizar.
a=open("res20.txt", "r")
a1 = a.readlines()
a.close()
resid=list()
for x in a1:
    resid.append((x.split("\n"))[0])
resid.pop(0)

#Tambien debo tener la estructura y las trayectorias.
#aa_ref = "3"
estructura = "cerrada-lig-rot.gro" #nombre del archivo de la estructura
trayectoria = "cerrada-lig-rot.xtc" #nomrbre del archivo de la trayectoria

In [544]:
#Creo el script para el primer analisis de puentes de hidrogeno. 
scr = open("hb1.tcl", "w") 
scr.write("mol new "+estructura+" type {gro} first 0 last -1 step 1 waitfor 1 \n")
scr.write("mol addfile "+trayectoria+" type {xtc} first 0 last -1 step 1 waitfor 2503 0 \n")
scr.write("\n")
scr.write("#using hbonds package\n")
scr.write("package require hbonds\n")
scr.write("\n")
for x in resid:
    scr.write('set sel1 [atomselect top "resid '+str(x)+'"]\n')
    scr.write('set sel2 [atomselect top "all and not resid '+str(x)+'"]\n')
    scr.write('hbonds -sel1 $sel1 -sel2 $sel2 -dist 3.5 -ang 30 -writefile yes -plot no -outfile hbonds'+x+'.dat -polar yes -type all -detailout hbonds-details'+x+'.dat -frames 500:2500 \n')
    scr.write("\n")
scr.write("exit")
scr.close()

In [545]:
sub.call(["vmd", "-startup", "hb1.tcl"]) 

0

In [546]:
#Diccionario donde las keys son los residuos que fueron maximos locales, y los values van a ser los 
#residuos con los que estos hagan puentes de hidrogeno. 
hbonds = dict()
aa = ("ALA","ARG","ASN","ASP","CYS", "GLN","GLY","GLU","HIS","ILE","LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL","1SY")
for i in resid:
    val = list()
    #A partir de aca lee el archivo detail.dat
    fin = open("hbonds-details"+i+".dat", "rt")
    data = fin.read()
    #data = data.replace(aa, '') ##tengo que ver como manejar esto 
    for x in aa:
        data = data.replace(str(x),'')
    fin.close()
    fin = open("hbonds-details"+i+".dat", "wt")
    fin.write(data)
    fin.close()
    f=open("hbonds-details"+i+".dat", "r")
    f1 = f.readlines()
    f.close()
    for j in range(len(f1)):
        if j > 1:
            g = f1[j].split("\t ")
            b = [ entry.split('-') for entry in g ]
            if b[0][0]!= i: 
                val.append(b[0][0])
            else:
                val.append(b[1][0]) 
    hbonds.update({i: val})

In [547]:
#Creo el script para el segundo analisis de puentes de hidrogeno. 
hb_oc = dict()
aux = ()

scr2 = open("hb2.tcl", "w") 
scr2.write("mol new "+estructura+" type {gro} first 0 last -1 step 1 waitfor 1 \n")
scr2.write("mol addfile "+trayectoria+" type {xtc} first 0 last -1 step 1 waitfor 2502 0 \n")
scr2.write("\n")
scr2.write("\n")
scr2.write("#using hbonds package\n")
scr2.write("package require hbonds\n")
scr2.write("\n")


for i in hbonds:
  reslist = hbonds[i]
  for j in range(len(reslist)):
    aux = (i, reslist[j])
    hb_oc.update({aux:0})
    scr2.write('set sel1 [atomselect top "resid '+i+'"]\n')
    scr2.write('set sel2 [atomselect top "resid '+reslist[j]+'"]\n')
    scr2.write("hbonds -sel1 $sel1 -sel2 $sel2 -dist 3.5 -ang 30 -writefile yes -plot no -outfile hbonds"+i+"-"+reslist[j]+".dat -polar yes -type all -detailout hbonds-details"+i+"-"+reslist[j]+".dat -frames 500:2500\n")
    
scr2.write("exit")
scr2.close()

In [548]:
sub.call(["vmd", "-startup", "hb2.tcl"]) 

0

In [549]:
#Escribo en un archivo la lista de puentes de hidrogeno 
new = open("hbonds_res.txt", "a")
new.write("Donor\tAceptor\tOcupación (%)\n")
for i in hb_oc:
    h=open("hbonds-details"+i[0]+"-"+i[1]+".dat", "r")
    h1 = h.readlines()
    h.close()
    for x in range(len(h1)):
        if x > 1:
            new.write(h1[x])
new.close()

In [550]:
os.remove('hb1.tcl')
os.remove('hb2.tcl')

In [551]:
import pandas as pd 
data = pd.read_csv("hbonds_res.txt", sep="\t")
data
data.drop_duplicates().to_csv("hbonds_res.csv")
data.drop_duplicates()

,Donor,Aceptor,Ocupación (%)
0,GLU151-Main,VAL147-Main,22.89%
1,SER150-Main,VAL147-Main,1.00%
2,SER150-Side,VAL147-Main,0.35%
3,LEU152-Main,GLU148-Main,54.17%
4,GLU151-Main,GLU148-Main,12.14%
...,...,...,...
2912,GLN539-Main,GLN538-Side,0.85%
2913,GLN539-Side,GLN538-Main,1.00%
2914,GLN538-Side,GLN539-Side,0.20%
2915,GLN538-Main,GLN539-Side,0.05%
